## E.2 MLE for Gaussian AR(1)-GARCH(1,1)

Fit a Gaussian AR(1)-GARCH(1,1) to the 10-year government bond yield. Use the following procedure:

1. Write a function, called "garch11_variance(alpha_0, alpha_1, beta_1, sigma2_1, epsilon)". It takes the parameters of the variance equation as an input as well as the residuals of the mean equation. The function returns the GARCH(1,1) implied variance. Note, the first variance is computed using "epsilon[0]" and the start value of the variance, i.e. "sigma2_1". 

2. Write a second function, called, "Neg_loglikelihood_ar1_Garch11(parameters)". It takes the model parameters as input and returns the negative joint log likelihood function. 

3. Use smart starting values for the optimization (from last week's Python for Financial Data Science material, see below). In addition, we use rather uninformative starting values for beta and sigma2_1, namely 0.01 and 1, respectively. **Praktomat: estimated parameters from local unconstrained optimization**

4. You want to use a bounded optimizer to ensure the estimate imply: (i) stationary interest rates (stationary mean equation), (ii) positive unconditional interest rates, (iii) stationary variance of interest rates (stationary variance equation), (iv) positive unconditional variance of interest rate. **Type of optimizer: differential_evolution**. **Praktomat: estimated parameter global constrained optimization**

5. Hand-in the mathematical algorithm and pseudo code that underlines your Python implementation. 

**Data**

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

y_df = pd.read_excel("Data/GovBondYields.xls", index_col = [0])
#10-yr maturity rate
y10_df = y_df.iloc[:,7]

**LL**

$$
L_T(\phi_0, \phi_1, \alpha_0, \alpha_1, \beta_1, \sigma_1) = \prod_{t=2}^T \frac{1}{\sqrt{ 2 \pi (\alpha_0 + \alpha_1 \epsilon^2_{t-1} + \beta_1 \sigma^2_{t-1})}} \times \exp\left( -\frac{(r_t - [\phi_0 + \phi_1 r_{t-1}])^2}{2 (\alpha_0 + \alpha_1 \epsilon^2_{t-1}+ \beta_1 \sigma^2_{t-1})} \right)
$$

with $\sigma^2_t = \alpha_0 + \alpha_1 \epsilon^2_{t-1} + \beta_1 \sigma^2_{t-1}, s.t. \sigma^2_1 = \text{known parameter}$

Note:
 $$
 \ln (L_T(.)) = \sum_{t=2}^T -\frac{1}{2} \ln(2\pi [\alpha_0 + \alpha_1 \epsilon^2_{t-1}+ \beta_1 \sigma^2_{t-1}]) - \frac{1}{2}  \frac{(r_t - [\phi_0 + \phi_1 r_{t-1}])^2}{2 (\alpha_0 + \alpha_1 \epsilon^2_{t-1}+ \beta_1 \sigma^2_{t-1})} 
 $$

**function GARCH_11_VARIANCE**

def garch11_variance(alpha_0, alpha_1, beta_1, sigma2_1, epsilon):
    var = np.zeros(len(epsilon))
    var[0] = alpha_0 + alpha_1 * epsilon[0]**2 + beta_1 * sigma2_1
    for i in range(1, )
        var[1:] = alpha_0 + alpha_1 * epsilon[:-1]**2 + beta_1 * var[i-1]
    return var

In [6]:
def garch11_variance(alpha_0, alpha_1, beta_1, sigma2_1, epsilon):
    var_arch = alpha_0 + alpha_1 * (epsilon[:-1])**2
    var = [sigma2_1]
    for i in range(0,len(var_arch)):
        var.append(var_arch[i] + beta_1 * var[i])
    var.pop(0)
    return np.array(var)

**-ln(L_T)**

In [7]:
def Neg_loglikelihood_ar1_Garch11(parameters, r_t=y10_df):
    phi_0, phi_1, alpha_0, alpha_1, beta_1, sigma2_1 = parameters
    
    means = phi_0 + phi_1 * r_t.iloc[:-1].values
    eps = r_t.iloc[1:].values - means
    var = garch11_variance(alpha_0, alpha_1, beta_1, sigma2_1, eps)

    loglikelihood =  np.sum(-0.5 * np.log(2 * np.pi * var) - ((y10_df.iloc[2:].values - means[1:])**2) / (2 * var))
    
    return -loglikelihood

**Start Values from 2pass Estimation**

In [8]:
#start values for AR(1)-GARCH(1,1) parameters from last week's Python for Financial Data Science material
phi0_start = 0.0204
phi1_start = 0.9962
alpha0_start = 0.0004
alpha1_start = 0.3157
#uninformative start values for GARCH part
beta1_start = 0.01
sigma2_1_start = 1

ar1_garch11_params_start = [phi0_start, phi1_start, alpha0_start, alpha1_start, beta1_start, sigma2_1_start]

**Unconstrained Optimization: Nelder-Mead Optimization**

In [9]:
import scipy.optimize as sco

ar1_garch11_params_optimal = sco.minimize(Neg_loglikelihood_ar1_Garch11, ar1_garch11_params_start, args=(y10_df),
                                         method='Nelder-Mead', options={'disp': True})

<ipython-input-7-3ea048519d69>:8: RuntimeWarning: invalid value encountered in log
  loglikelihood =  np.sum(-0.5 * np.log(2 * np.pi * var) - ((y10_df.iloc[2:].values - means[1:])**2) / (2 * var))


Optimization terminated successfully.
         Current function value: 166.840529
         Iterations: 651
         Function evaluations: 1042


**Output:**

In [10]:
#unconstrained
#phi_0, phi_1, alpha_0, alpha_1, beta_1, sigma2_1
ar1_garch11_params_optimal.x

array([-9.40566161e-02,  1.02268374e+00,  6.73421893e-03,  4.44696040e+00,
        3.77398370e-03, -1.90019306e+01])

**Stationary GARCH(1,1)** Stationary Conditions and Positivity Restrictions for the Variance:

Stationary mean equation:
$$
|\phi_1| < 1
$$

Stationary variance equation:
$$
\alpha_1 + \beta_1 < 1
$$

Positive unconditional variance forecast:
$$
\alpha_0 > 0 \qquad \text{and} \qquad \alpha_1, \beta_1, \sigma^2_{1} \in \mathcal{R}_+
$$

Positive unconditional interest rates:
$$
\phi_0 > 0, \qquad \phi_1 > 0
$$


**Bounded Optimization:** 

Hints:

1. Please specify the bounds for all the parameters.

2. For inequality constraints please following the doc from scipy in the following link:
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html. In addition, for $ \alpha_1 + \beta_1 < 1$ we specify it as $ 0<\alpha_1 + \beta_1 < 1$. 

3. Please use seed=1

In [11]:
bounds = ((-5,5),
          (-5,5),
          (-5,5),
          (-5,5),
          (-5,5),
          (-5,5))
#phi_0, phi_1, alpha_0, alpha_1, beta_1, sigma2_1

def stat_mean_equ(parameters):
    return abs(parameters[1])

def stat_var_equ(parameters):
    return parameters[3] + parameters[4]

def pos_uncon_var_forecast(parameters):
    return parameters[2], parameters[3], parameters[4], parameters[5]

def pos_uncon_interst_rates(parameters):
    return parameters[0], parameters[1]
    


nlc1 = sco.NonlinearConstraint(stat_mean_equ, 0, 1)
nlc2 = sco.NonlinearConstraint(stat_var_equ, 0, 1)
nlc3 = sco.NonlinearConstraint(pos_uncon_var_forecast, 0, np.inf)
nlc4 = sco.NonlinearConstraint(pos_uncon_interst_rates, 0, np.inf)

In [12]:
constraint_est = sco.differential_evolution(Neg_loglikelihood_ar1_Garch11, bounds,args=(y10_df, ), seed=1,
                                            constraints=(nlc1, nlc2, nlc3, nlc4))

C:\Users\Luca\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


In [13]:
#constrained
#phi_0, phi_1, alpha_0, alpha_1, beta_1, sigma2_1
constraint_est.x

array([4.95223065e-02, 9.91773656e-01, 2.26752057e-04, 1.44879665e-01,
       8.55120334e-01, 2.80553420e-03])